## 한국무역통계정보포털 크롤링

In [1]:
# Module
import glob
import pandas as pd
import numpy as np
import re

from bs4 import BeautifulSoup
from urllib.request import urlopen

import random
import time
import math

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def data_scraping(max_row):
    rawitems_list = list()
    for i in range(1,max_row+1):
        item_dict = dict()
        for j in range(1,8):
            item_path = """//*[@id="{}"]/td[{}]""".format(i,j)
            item = driver.find_element_by_xpath(item_path).get_attribute('title')
            item_dict[col_list[j-1]] = item
        df_rawitem = pd.DataFrame(item_dict, [0])
        rawitems_list.append(df_rawitem)

    df_raw = pd.concat(rawitems_list).reset_index(drop=True)
    return df_raw

### LOGIN

In [7]:
# Login Page
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.bandtrass.or.kr/login.do?returnPage=M')

# driver = webdriver.Chrome('/Users/hyungkeun/chromedriver')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/hyungkeun/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [8]:
# Input login info
time.sleep(random.random()+random.random())
xpath_id = """//*[@id="id"]"""
driver.find_element_by_xpath(xpath_id).click()
elem_login = driver.find_element_by_id('id')
elem_login.clear()
elem_login.send_keys('***')

time.sleep(random.random()+random.random())
xpath_pw = """//*[@id="pw"]"""
driver.find_element_by_xpath(xpath_pw).click()
elem_login = driver.find_element_by_id('pw')
elem_login.clear()
elem_login.send_keys("***")

# Entry
time.sleep(random.random()+random.random())
xpath_entry = """//*[@id="page-wrapper"]/div/div/div[2]/div/table/tbody/tr[1]/td[2]/button"""
driver.find_element_by_xpath(xpath_entry).click()
time.sleep(random.random()+random.random())


# Change password next time
time.sleep(random.random()+random.random()+0.54321)
xpath_nochg = """//*[@id="pass_change"]/div/div/div[2]/button[2]"""
driver.find_element_by_xpath(xpath_nochg).click()

In [9]:
# Column head
num = '순번'
goodcd = '품목코드'
goodnm = '품목명'
nation = '국가명'
amt_dollar = '금액(달러)'
amt_won = '금액(원화)'
net_wght = '중량(Kg)'
col_list = [num, goodcd, goodnm, nation, amt_dollar, amt_won, net_wght]
col_list

['순번', '품목코드', '품목명', '국가명', '금액(달러)', '금액(원화)', '중량(Kg)']

In [10]:
# goods list
goods_cd_list = ['240220','1902301010', '1901101010', '3304', '3304991000',
                 '3304992000', '330410', '330420', '3307909000']
print(len(goods_cd_list))

9


### CRAWL DATA

In [12]:
all_dfraw_list = list()


for good in goods_cd_list:
    # Go to the page that has data
    driver.get('https://www.bandtrass.or.kr/index.do')
    time.sleep(random.randint(5, 7)+random.random())
    driver.get('https://www.bandtrass.or.kr/customs/total.do?command=CUS001View&viewCode=CUS00401')
    time.sleep(random.randint(30, 35)+random.random())
    
    # Select multiple goods
    time.sleep(random.randint(10, 15)+random.random())
    xpath = """//*[@id="tr1"]/td/div[2]/label"""
    driver.find_element_by_xpath(xpath).click()
    time.sleep(random.random()+random.random())
    
    # Setting code
    time.sleep(random.randint(5, 7)+random.random())
    xpath = """//*[@id="SelectCd"]"""
    driver.find_element_by_xpath(xpath).click()
    elem_good = driver.find_element_by_id('SelectCd')
    elem_good.clear()
    elem_good.send_keys(good)

    # Entry
    time.sleep(random.randint(5, 7)+random.random())
    xpath = """//*[@id="form"]/div/div[1]/div[3]/button"""
    driver.find_element_by_xpath(xpath).click()
    
    # Waiting for loading data. It might take long but can't be helped
    # I randomly set 35~40 seconds for safety    
    time.sleep(random.randint(30, 35)+random.random())

    # Soup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
        
    # Page Info
    totcnt = int(soup.find_all('strong', class_='totCnt')[0].get_text())
    page_nums = math.ceil(totcnt/50)
    page_remain = totcnt%50
    
    
    # SCRAPING
    if totcnt >= 50:
        for page_num in range(0,page_nums):      
            if page_nums == 1:
                time.sleep(random.randint(5, 7)+random.random())
                result = data_scraping(50)
                all_dfraw_list.append(result)
                print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')

            elif page_nums == 2:
                if page_num == 0:
                    result = data_scraping(50)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                elif page_num == page_nums-1:
                    x_path = """//*[@id="form"]/div/div[4]/div/span/a[{}]""".format(page_num)
                    driver.find_element_by_xpath(x_path).click()
                    time.sleep(random.randint(30, 35)+random.random())
                    result = data_scraping(page_remain)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)

    #                 # return to Page 1
    #                 time.sleep(random.randint(5, 10)+random.random())
    #                 x_path = """//*[@id="form"]/div/div[4]/div/span/a[1]"""
    #                 driver.find_element_by_xpath(x_path).click()
    #                 time.sleep(random.randint(35, 40)+random.random())        
            elif page_nums == 3:
                if page_num == 0:
                    result = data_scraping(50)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)
                elif page_num == 1:
                    x_path = """//*[@id="form"]/div/div[4]/div/span/a[1]"""
                    driver.find_element_by_xpath(x_path).click()
                    time.sleep(random.randint(30, 35)+random.random())
                    result = data_scraping(50)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)
                else:
                    x_path = """//*[@id="form"]/div/div[4]/div/span/a[2]"""
                    driver.find_element_by_xpath(x_path).click()
                    time.sleep(random.randint(30, 35)+random.random())
                    result = data_scraping(page_remain)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)

            else:
                if page_num == 0:
                    result = data_scraping(50)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)
                elif page_num == page_nums-1:
                    x_path = """//*[@id="form"]/div/div[4]/div/span/a[{}]""".format(page_num)
                    driver.find_element_by_xpath(x_path).click()
                    time.sleep(random.randint(30, 35)+random.random())
                    result = data_scraping(page_remain)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)
                else:
                    x_path = """//*[@id="form"]/div/div[4]/div/span/a[{}]""".format(page_num)
                    driver.find_element_by_xpath(x_path).click()
                    time.sleep(random.randint(30, 35)+random.random())
                    result = data_scraping(50)
                    all_dfraw_list.append(result)
                    print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')
                    time.sleep(1)

    else:
        time.sleep(random.randint(5, 7)+random.random())
        result = data_scraping(totcnt)
        all_dfraw_list.append(result)
        print('page {} of {} in cd:{} is successfully finished'.format(page_num+1, page_nums, good), end='\n')


page 1 of 2 in cd:240220 is successfully finished
page 2 of 2 in cd:240220 is successfully finished
page 1 of 3 in cd:1902301010 is successfully finished
page 2 of 3 in cd:1902301010 is successfully finished
page 3 of 3 in cd:1902301010 is successfully finished
page 3 of 1 in cd:1901101010 is successfully finished
page 1 of 4 in cd:3304 is successfully finished
page 2 of 4 in cd:3304 is successfully finished
page 3 of 4 in cd:3304 is successfully finished
page 4 of 4 in cd:3304 is successfully finished
page 1 of 4 in cd:3304991000 is successfully finished
page 2 of 4 in cd:3304991000 is successfully finished
page 3 of 4 in cd:3304991000 is successfully finished
page 4 of 4 in cd:3304991000 is successfully finished
page 1 of 3 in cd:3304992000 is successfully finished
page 2 of 3 in cd:3304992000 is successfully finished
page 3 of 3 in cd:3304992000 is successfully finished
page 1 of 3 in cd:330410 is successfully finished
page 2 of 3 in cd:330410 is successfully finished
page 3 of 3 in

In [13]:
all_df = pd.concat(all_dfraw_list).reset_index(drop=True)
print(all_df.shape)
all_df.head(3)

(1003, 7)


,순번,품목코드,품목명,국가명,금액(달러),금액(원화),중량(Kg)
0,1,240220,궐련(담배를 함유한 것으로 한정한다),아랍에미리트 연합,"6,384,755","7,825,985,208","304,704.00"
1,2,240220,궐련(담배를 함유한 것으로 한정한다),일본,"5,590,602","6,833,755,540","391,651.93"
2,3,240220,궐련(담배를 함유한 것으로 한정한다),미국,"3,349,624","4,076,329,629","219,892.81"


In [14]:
# all_df.to_excel('/Users/hyungkeun/Desktop/...xlsx', index=False)